In [1]:
%%time
#init
#set path
from os import listdir #os no need to install
import pandas as pd
import numpy as np
import math
from pyspark.sql import SparkSession
from pyspark.sql.functions import sqrt
from pyspark.sql import functions as F
from pyspark.sql import types

mypath = "D:/Download/NYtaxi/headdata" #your dir
files = listdir(mypath)
df = ['PU_Time','DO_Time','Trip_Distance','Rate_Code','PU_Lon','PU_Lat','DO_Lon','DO_Lat','Fare_Amt','Extra','Tip_Amt','PULocationID','DOLocationID']

PU_Time = ('Pickup_date','Pickup_DateTime','pickup_datetime','lpep_pickup_datetime','Trip_Pickup_DateTime','tpep_pickup_datetime')
DO_Time = ('DropOff_datetime','dropoff_datetime','Lpep_dropoff_datetime','lpep_dropoff_datetime','Trip_Dropoff_DateTime','tpep_dropoff_datetime')
Trip_Distance = ('Trip_distance','trip_distance','Trip_Distance')
Rate_Code = ('rate_code','RateCodeID','Rate_Code','RatecodeID')
PU_Lon = ('Pickup_longitude','Start_Lon','pickup_longitude')
PU_Lat = ('Pickup_latitude','Start_Lat','pickup_latitude')
DO_Lon = ('Dropoff_longitude','End_Lon','dropoff_longitude')
DO_Lat = ('Dropoff_latitude','End_Lat','dropoff_latitude')
Fare_Amt = ('Fare_amount','fare_amount','Fare_Amt')
Extra = ('Extra','extra','surcharge')
Tip_Amt = ('Tip_amount','tip_amount','Tip_Amt')
PULocationID = ('PULocationID')
DOLocationID = ('DOLocationID')


cols = [PU_Time,DO_Time,Trip_Distance,Rate_Code,PU_Lon,PU_Lat,DO_Lon,DO_Lat,Fare_Amt,Extra,Tip_Amt,PULocationID,DOLocationID]

Wall time: 364 ms


In [2]:
%%time
#infile
spark = SparkSession\
        .builder\
        .appName("moz")\
        .getOrCreate()
print('目錄下的檔案有:')
t=[]
for  f in files:
    print(f)
    fullpath = mypath+'/'+f
    t += [spark.read.csv(fullpath,sep = ',',header = True,inferSchema = True).limit(49999)]
df0=t[0]
print('第一個檔案的原始欄位')
print(df0.count(),len(df0.columns))
print('---------')
print(df0.dtypes)

目錄下的檔案有:
2009_Yellow_head_50K_y.csv
2010_Yellow_head_50K_y.csv
2011_Yellow_head_50K_y.csv
2012_Yellow_head_50K_y.csv
2013_Green_head_50K_x.csv
2013_Yellow_head_50K.csv
2014_Green_head_50K.csv
2014_Yellow_head_50K_x.csv
2015_0106_Yellow_head_50K.csv
2015_Green_head_50K.csv
2016_Green_head_50K.csv
2016_Yellow_head_50K.csv
2017_Green_head_50K.csv
2017_Yellow_head_50K.csv
第一個檔案的原始欄位
49999 18
---------
[('vendor_name', 'string'), ('Trip_Pickup_DateTime', 'timestamp'), ('Trip_Dropoff_DateTime', 'timestamp'), ('Passenger_Count', 'int'), ('Trip_Distance', 'double'), ('Start_Lon', 'double'), ('Start_Lat', 'double'), ('Rate_Code', 'string'), ('store_and_forward', 'int'), ('End_Lon', 'double'), ('End_Lat', 'double'), ('Payment_Type', 'string'), ('Fare_Amt', 'double'), ('surcharge', 'double'), ('mta_tax', 'string'), ('Tip_Amt', 'double'), ('Tolls_Amt', 'double'), ('Total_Amt', 'double')]
Wall time: 14.8 s


In [3]:
%%time
#change columns name
for j in range(0,len(t)):
    for w in range(0,len(t[j].columns)):
        for i in range(0,len(cols)):
            for k in range(0,len(cols[i])):
                if t[j].columns[w] == cols[i][k]:
                    t[j]=t[j].withColumnRenamed(t[j].columns[w],df[i])
print('第一個檔案的修正欄位')
print(t[0].columns)

第一個檔案的修正欄位
['vendor_name', 'PU_Time', 'DO_Time', 'Passenger_Count', 'Trip_Distance', 'PU_Lon', 'PU_Lat', 'Rate_Code', 'store_and_forward', 'DO_Lon', 'DO_Lat', 'Payment_Type', 'Fare_Amt', 'Extra', 'mta_tax', 'Tip_Amt', 'Tolls_Amt', 'Total_Amt']
Wall time: 530 ms


In [4]:
%%time
#Drop useless columns
for i in range(len(t)):
    for c in t[i].columns:
        if c not in df:
            t[i] = t[i].drop(c)
print('第一個檔案的刪除後欄位')
print(t[0].columns)
print('每個table的欄位數')
for tb in t:
    print(len(tb.columns),end=', ')

第一個檔案的刪除後欄位
['PU_Time', 'DO_Time', 'Trip_Distance', 'PU_Lon', 'PU_Lat', 'Rate_Code', 'DO_Lon', 'DO_Lat', 'Fare_Amt', 'Extra', 'Tip_Amt']
每個table的欄位數
11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 13, 13, 9, 9, Wall time: 338 ms


In [9]:
%%time
#Check NA values
for i in range(0,len(t)):
    for col in df:
        if col not in t[i].columns:
            print('Table',files[i],'do not have column', col)
        else:
            tmp=t[i].filter(t[i][col].isNull()).count()
            if tmp!=0:
                print('In table',files[i],'column',col,'has',tmp,'NA values.')
    print('---------------------------------------------')

Table 2009_Yellow_head_50K_y.csv do not have column Rate_Code
Table 2009_Yellow_head_50K_y.csv do not have column PULocationID
Table 2009_Yellow_head_50K_y.csv do not have column DOLocationID
---------------------------------------------
Table 2010_Yellow_head_50K_y.csv do not have column PULocationID
Table 2010_Yellow_head_50K_y.csv do not have column DOLocationID
---------------------------------------------
Table 2011_Yellow_head_50K_y.csv do not have column PULocationID
Table 2011_Yellow_head_50K_y.csv do not have column DOLocationID
---------------------------------------------
Table 2012_Yellow_head_50K_y.csv do not have column PULocationID
Table 2012_Yellow_head_50K_y.csv do not have column DOLocationID
---------------------------------------------
Table 2013_Green_head_50K_x.csv do not have column PULocationID
Table 2013_Green_head_50K_x.csv do not have column DOLocationID
---------------------------------------------
Table 2013_Yellow_head_50K.csv do not have column PULocation

In [5]:
%%time
#Handle NA values

#Drop all NA columns
for i in range(0,len(t)):
    for col in t[i]:
        if t[i].filter(col.isNull()).count()==t[i].count():
            t[i] = t[i].drop(col)

#Drop few NA rows
for i in range(0,len(t)):
     t[i] = t[i].na.drop()

Wall time: 29.4 s


In [6]:
%%time
#New Columns


# timeDiff = (F.unix_timestamp('EndDateTime')
#             - F.unix_timestamp('StartDateTime'))
# df = df.withColumn("Duration", timeDiff)

#True_Distance
for i in range(len(t)):
    if 'DO_Lon' in t[i].columns:
        t[i] = t[i].withColumn('True_Distance',(t[i].DO_Lon-t[i].PU_Lon)*(t[i].DO_Lon-t[i].PU_Lon)+(t[i].DO_Lat-t[i].PU_Lat)*(t[i].DO_Lat-t[i].PU_Lat))
for i in range(len(t)):
    if 'True_Distance' in t[i].columns:
        t[i] = t[i].withColumn('True_Distance',sqrt(t[i].True_Distance))
#Trip_Time
for i in range(len(t)):
    if 'DO_Time' in t[i].columns:
        if i in (4, 6, 9, 10, 11, 12, 13):
            tt = F.unix_timestamp(t[i].DO_Time,'MM/dd/yyyy h:mm:ss a')-F.unix_timestamp(t[i].PU_Time,'MM/dd/yyyy h:mm:ss a')
            t[i] = t[i].withColumn('Trip_Time', tt) 
        else:
            tt = F.unix_timestamp(t[i].DO_Time)-F.unix_timestamp(t[i].PU_Time)
            t[i] = t[i].withColumn('Trip_Time', tt) 
        

Wall time: 245 ms


In [12]:
#PU_Year
#PU_Month
#PU_Day
#PU_Hour
#PU_Minute
#PU_Second
#PU_WeekDay(0:6)
#Trip_Distance :miles to km
#Speed(km/hr)
for i in range(len(t)):
    t[i] = t[i].withColumn('PU_Year',F.year(t[i].PU_Time))
    t[i] = t[i].withColumn('PU_Month',F.month(t[i].PU_Time))
    t[i] = t[i].withColumn('PU_Day',F.dayofmonth(t[i].PU_Time))
    t[i] = t[i].withColumn('PU_Hour',F.hour(t[i].PU_Time))
    t[i] = t[i].withColumn('PU_Minute',F.minute(t[i].PU_Time))
    t[i] = t[i].withColumn('PU_Second',F.second(t[i].PU_Time))
    t[i] = t[i].withColumn('PU_WeekDay',F.dayofweek(t[i].PU_Time))
    t[i] = t[i].withColumn('Trip_Distance',t[i].Trip_Distance*1.609344)
    t[i] = t[i].withColumn('Speed',t[i].Trip_Distance*3600/t[i].Trip_Time)
print(t[0].columns)
print(t[0].head())

['PU_Time', 'DO_Time', 'Trip_Distance', 'PU_Lon', 'PU_Lat', 'DO_Lon', 'DO_Lat', 'Fare_Amt', 'Extra', 'Tip_Amt', 'True_Distance', 'Trip_Time', 'PU_Year', 'PU_Month', 'PU_Day', 'PU_Hour', 'PU_Minute', 'PU_Second', 'PU_WeekDay', 'Speed']
Row(PU_Time=datetime.datetime(2009, 1, 4, 2, 52), DO_Time=datetime.datetime(2009, 1, 4, 3, 2), Trip_Distance=4.23257472, PU_Lon=-73.991957, PU_Lat=40.721567, DO_Lon=-73.993803, DO_Lat=40.695922, Fare_Amt=8.9, Extra=0.5, Tip_Amt=0.0, True_Distance=0.025711354320608257, Trip_Time=600, PU_Year=2009, PU_Month=1, PU_Day=4, PU_Hour=2, PU_Minute=52, PU_Second=0, PU_WeekDay=1, Speed=25.39544832)


In [13]:
#Trip_Time<=0
#Trip_Distance<=0
#'Fare_Amt', 'Extra', 'Tip_Amt'<0
#Trip_Distance<True_Distance
#True_Distance>=5
for i in range(len(t)):
    print(files[i])
    tttmp=0
    tmp0 = t[i].filter(t[i].Trip_Time<=0).count()
    tmp1 = t[i].filter(t[i].Trip_Distance<=0).count()
    tmp2 = t[i].filter(t[i].Fare_Amt<0).count()
    tmp3 = t[i].filter(t[i].Extra<0).count()
    tmp4 = t[i].filter(t[i].Tip_Amt<0).count()
    if tmp0>0 :
        tttmp+=tmp0
        print('Trip_Time<=0 的資料有',tmp0,'筆')
    if tmp1>0 :
        tttmp+=tmp1
        print('Trip_Distance<=0 的資料有',tmp1,'筆')
    if tmp2>0 :
        tttmp+=tmp2
        print('Fare_Amt<0 的資料有',tmp1,'筆')
    if tmp3>0 :
        tttmp+=tmp3
        print('df0.Extra<0 的資料有',tmp3,'筆')
    if tmp4>0 :
        tttmp+=tmp4
        print('Tip_Amt<0 的資料有',tmp4,'筆')
        
    if 'True_Distance' in t[i].columns:
        tt0 = t[i].filter(t[i].Trip_Distance<=t[i].True_Distance).count()
        if tt0>0 :
            tttmp+=tt0
            print('Trip_Distance<=True_Distance 的資料有',tt0,'筆')
        tt = t[i].filter(t[i].True_Distance>=5).count()
        if tt>0 :
            tttmp+=tt
            print('True_Distance>=5 的資料有',tt,'筆')
    print()
    print('Total:',tttmp)
    print('---------------------------------')

2009_Yellow_head_50K_y.csv
Trip_Time<=0 的資料有 1485 筆
Trip_Distance<=0 的資料有 403 筆
Trip_Distance<=True_Distance 的資料有 433 筆
True_Distance>=5 的資料有 27 筆

Total: 2348
---------------------------------
2010_Yellow_head_50K_y.csv
Trip_Time<=0 的資料有 125 筆
Trip_Distance<=0 的資料有 274 筆
Trip_Distance<=True_Distance 的資料有 297 筆
True_Distance>=5 的資料有 23 筆

Total: 719
---------------------------------
2011_Yellow_head_50K_y.csv
Trip_Time<=0 的資料有 33 筆
Trip_Distance<=0 的資料有 409 筆
Trip_Distance<=True_Distance 的資料有 438 筆
True_Distance>=5 的資料有 47 筆

Total: 927
---------------------------------
2012_Yellow_head_50K_y.csv
Trip_Time<=0 的資料有 22 筆
Trip_Distance<=0 的資料有 251 筆
Trip_Distance<=True_Distance 的資料有 308 筆
True_Distance>=5 的資料有 66 筆

Total: 647
---------------------------------
2013_Green_head_50K_x.csv
Trip_Time<=0 的資料有 61 筆
Trip_Distance<=0 的資料有 1493 筆
Trip_Distance<=True_Distance 的資料有 1543 筆
True_Distance>=5 的資料有 108 筆

Total: 3205
---------------------------------
2013_Yellow_head_50K.csv
Trip_Time<=0 

In [14]:
#Handle nonsense values
for j in range(0,len(t)):
    t[j]=t[j].filter(t[j]['Trip_Time']>0)
    t[j]=t[j].filter(t[j].Trip_Distance>0)
    t[j]=t[j].filter(t[j].Fare_Amt>=0)
    t[j]=t[j].filter(t[j].Extra>=0)
    t[j]=t[j].filter(t[j].Tip_Amt>=0)
    if 'True_Distance' in t[j].columns:
        t[j]=t[j].filter(t[j].True_Distance<5)
        t[j]=t[j].filter(t[j].Trip_Distance>t[j].True_Distance)
for j in range(0,len(t)):        
    print(t[j].count(),len(t[j].columns))

48283 20
49635 21
49554 21
49686 21
48452 21
49611 21
48926 21
49725 21
49535 21
49184 21
49208 21
49687 18
49286 18
49583 18


In [17]:
#Check Outliers
#Speed ,Fare Outliers
needCheck=['Speed','Fare_Amt','Extra','Tip_Amt']
def detect_outlier(df,cols):
    quan = df.approxQuantile(col=cols,probabilities=[0.25,0.75],relativeError=0)
    print(quan[1])
#     tmp = df.filter(df[cols]<quan[0]).count()+df.filter(df[cols]>quan[1]).count()
#     if tmp!=0:
#         print(col,"的outliers 有",tmp,"筆")
for i in range(len(t)):
    for j in needCheck:
#         detect_outlier(t[i],j)
        quan = t[i].approxQuantile(col=j,probabilities=[0.25,0.75],relativeError=0)
        print(quan)

[14.58065664, 27.0369792]
[5.7, 10.9]
[0.0, 0.5]
[0.0, 0.0]
[14.966899200000002, 25.72908409199048]
[5.3, 10.5]
[0.0, 0.5]
[0.0, 1.0]
[12.573839599332223, 22.7498890052356]
[5.7, 10.9]
[0.0, 0.5]
[0.0, 1.11]
[14.148988380462724, 24.97952578581363]
[5.7, 10.9]
[0.0, 0.5]
[0.0, 1.62]
[16.031890285714287, 26.325297917710202]
[6.5, 15.5]
[0.0, 0.5]
[0.0, 1.1]
[17.743017599999998, 29.78524356923077]
[6.0, 13.0]
[0.0, 0.0]
[0.0, 1.7]
[16.428086200378072, 27.56999918422842]
[6.5, 15.0]
[0.0, 0.5]
[0.0, 0.0]
[15.836824783599088, 27.009969230769233]
[7.0, 14.5]
[0.5, 0.5]
[1.25, 2.9]
[16.037753356401385, 26.277841555075597]
[6.5, 15.5]
[0.5, 0.5]
[0.0, 2.3]
[15.122057814029366, 24.127554172323762]
[6.5, 15.5]
[0.0, 0.5]
[0.0, 2.0]
[17.121333689227836, 27.432000000000002]
[7.0, 16.5]
[0.0, 0.5]
[0.0, 1.96]
[11.798594917933132, 22.51556155450237]
[6.5, 15.0]
[0.0, 0.5]
[0.0, 2.36]
[17.2985411943128, 27.090624000000005]
[6.5, 15.0]
[0.0, 0.5]
[0.0, 1.76]
[16.92946285714286, 28.090368]
[6.5, 16.0]


In [ ]:
df0 = t[4]
# print(df0.approxQuantile(col='Fare_Amt',probabilities=[0.25,0.5,0.75],relativeError=0))
# type(df0.Speed)
# type(df0.select('Speed').collect())
print(df0.count(),len(df0.columns))